#**5 LOS Tabular**

##**5.1 Setup**

In [ ]:
# First install package from terminal:
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon  # autogluon==0.4.1
!pip install lime

  Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 60.2.0
    Uninstalling setuptools-60.2.0:
      Successfully uninstalled setuptools-60.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
openxlab 0.0.36 requires setuptools~=60.2.0, but you have setuptools 69.2.0 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.


  Using cached setuptools-60.2.0-py3-none-any.whl.metadata (5.1 kB)
Using cached setuptools-60.2.0-py3-none-any.whl (953 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.2.0
    Uninstalling setuptools-69.2.0:
      Successfully uninstalled setuptools-69.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.3 requires setuptools>65.5.1, but you have setuptools 60.2.0 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.


In [ ]:
# import all required modules
import pandas as pd
import numpy as np
import sklearn
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import cohen_kappa_score
from sklearn.inspection import permutation_importance
from autogluon.tabular import TabularPredictor
import torch
from transformers import BertTokenizerFast, BertModel, BertConfig, BertForPreTraining, BertForSequenceClassification, pipeline
import lime
import lime.lime_text
import pickle

In [1]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC Project/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()


print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
# path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
# path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"
# path_to_tables = f"{path_to_repo}tables"

# os.makedirs(path_to_tables, exist_ok = True)

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive
/content/gdrive/MyDrive/MIMIC Project/


##**5.2 Import the dataset**

In [ ]:
def load_datasets(method):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    """
    global path_to_processed
    # load it back
    train = pd.read_feather('{}{}train_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    test = pd.read_feather('{}{}test_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    return train, test
# compute other metrics
def perf_evaluator(y_test, y_pred, y_pred_proba):
    """ Function to display the main classification performance metrics """
    kappa = cohen_kappa_score(y_test, y_pred)
    precision, recall, prc_th = precision_recall_curve(y_test, y_pred_proba)
    prc_auc = auc(recall, precision)
    return kappa, prc_auc

In [ ]:
# PARAMETERS

session_seed = 42 # set seed for our session
seed_tag = f'_{session_seed}'

random.seed(session_seed)


# Set to True if we want to include deaths
death_incl = False
death_tag = np.where(death_incl,"_death", "")

##**5.3 Train the Tabular Model (Structured DataSet)**

### 5.3.1 los_cat_UF

In [ ]:
# Model Parameters
label = 'los_cat_UF'
metric = 'roc_auc'


# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_los_UF_{method}{death_tag}') ######                                                                                                   # add los_UF_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_los_UF_{preproc_tag}{lemma_tag}{method}{death_tag}'                                                                                 # add _los_UF_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['los_cat_UF'], y_pred, y_pred_proba)                                                                                                          # add_UF
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_los_UF_{method}{death_tag}.xlsx')                                                                                      # add los_UF_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_los_UF_{method}{death_tag}.xlsx')                                                                               # add los_UF_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_los_UF_{method}{death_tag}.xlsx')                                                                                 # add los_UF_

tabular
frequency


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_los_UF__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail:    


Test set performance:
{'roc_auc': 0.9562785307163097, 'accuracy': 0.9505923143600116, 'balanced_accuracy': 0.7379118341826282, 'mcc': 0.578860704093716, 'f1': 0.5899280575539568, 'precision': 0.743202416918429, 'recall': 0.48906560636182905}

Performance metrics:
{"Cohen's Kappa": 0.5648271072481756, 'PRC AUC': 0.6975893674815121}


Computing feature importance via permutation shuffling for 55 features using 5000 rows with 5 shuffle sets...



Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.956279   0.934789     roc_auc        0.429589       0.183006   
1              CatBoost    0.953212   0.929285     roc_auc        0.024662       0.011586   
2            LightGBMXT    0.951917   0.931724     roc_auc        0.051299       0.022760   
3              LightGBM    0.951702   0.929259     roc_auc        0.021201       0.009831   
4               XGBoost    0.950488   0.926099     roc_auc        0.049150       0.025971   
5        NeuralNetTorch    0.949319   0.929626     roc_auc        0.100040       0.050264   
6         LightGBMLarge    0.948368   0.923930     roc_auc        0.031935       0.012813   
7       NeuralNetFastAI    0.947832   0.922006     roc_auc        0.143584       0.049056   
8      RandomForestEntr    0.941400   0.915620     roc_auc        0.197751       0.103182   
9        ExtraTreesEntr    0.939741   0.909460    

...


Test set performance:
{'roc_auc': 0.9605891679057916, 'accuracy': 0.9533371857844554, 'balanced_accuracy': 0.7513016619089018, 'mcc': 0.6053963182907868, 'f1': 0.6159334126040428, 'precision': 0.7662721893491125, 'recall': 0.5149105367793241}

Performance metrics:
{"Cohen's Kappa": 0.5921084749918543, 'PRC AUC': 0.7180695301909735}


These features in provided data are not utilized by the predictor and will be ignored: ['F1', 'F2', 'F3', 'F4', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F13', 'F14', 'F15', 'F16', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F27', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39', 'F40', 'F41', 'F42', 'F43', 'F45', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F58', 'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F74', 'F75', 'F76', 'F77', 'F79', 'F80', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'F90', 'F92', 'F93', 'F94', 'F95', 'F96', 'F98', 'F99', 'F100', 'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110', 'F111', 'F112', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F121', 'F122', 'F123', 'F124', 'F126', 'F128', 'F130', 'F131', 'F132', 'F133', 'F134', 'F135', 'F136', 'F138', 'F139', 'F140', 'F141', 'F142', 'F143', 'F144', 'F146', 'F1


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.960589   0.940558     roc_auc        0.928483       0.405817   
1              LightGBM    0.958595   0.935984     roc_auc        0.032952       0.014254   
2              CatBoost    0.957112   0.933158     roc_auc        0.020204       0.013758   
3            LightGBMXT    0.957065   0.937323     roc_auc        0.041656       0.016003   
4         LightGBMLarge    0.955175   0.928918     roc_auc        0.031400       0.013606   
5        NeuralNetTorch    0.953778   0.935818     roc_auc        0.184254       0.087755   
6               XGBoost    0.953540   0.929401     roc_auc        0.060333       0.023251   
7       NeuralNetFastAI    0.949537   0.927706     roc_auc        0.146007       0.052937   
8        ExtraTreesEntr    0.944285   0.922003     roc_auc        0.253824       0.103740   
9      RandomForestEntr    0.944133   0.923309    

	419.19s	= Expected runtime (83.84s per shuffle set)
	180.15s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                    importance    stddev       p_value  n  p99_high   p99_low     percent
count_transfer        0.023394  0.000991  3.857176e-07  5  0.025434  0.021353  100.000000
urea_n_max            0.018253  0.001479  5.121442e-06  5  0.021297  0.015208   78.023571
platelets_min         0.012743  0.002062  7.955162e-05  5  0.016989  0.008496   54.469931
platelets_max         0.012197  0.001715  4.574394e-05  5  0.015729  0.008665   52.138321
urea_n_min            0.007968  0.000993  2.832804e-05  5  0.010012  0.005924   34.060265
platelets_mean        0.004131  0.000596  5.043230e-05  5  0.005358  0.002905   17.659781
temp_max              0.003883  0.000807  2.110646e-04  5  0.005544  0.002222   16.598798
F125                  0.003635  0.001207  1.264706e-03  5  0.006120  0.001151   15.539461
drg_severity          0.003312  0.001075  1.164892e-03  5  0.005526  0.001098   14.157318
resprate_min          0.001934  0.000533  6.289997e-04  5  0.003032  0.000836  

### 5.3.2 los_cat_80

In [ ]:
# Model Parameters
label = 'los_cat_80'
metric = 'roc_auc'


# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_los_80_{method}{death_tag}') ######                                                                                                   # add los_80_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_los_80_{preproc_tag}{lemma_tag}{method}{death_tag}'                                                                                 # add _los_80_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['los_cat_80'], y_pred, y_pred_proba)                                                                                                          # add_80
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_los_80_{method}{death_tag}.xlsx')                                                                                      # add los_80_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_los_80_{method}{death_tag}.xlsx')                                                                               # add los_80_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_los_80_{method}{death_tag}.xlsx')                                                                                 # add los_80_

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


tabular
frequency


Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_los_80__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail:       46.99 GB / 50.99 GB (92.2%)
Disk Space Avail:   10.49 GB / 100.00 GB (10.5%)
Train Data Rows:    27684
Train Data Columns: 55
Label Column:       los_cat_80
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping: 


Test set performance:
{'roc_auc': 0.943844238545248, 'accuracy': 0.8978618896272753, 'balanced_accuracy': 0.8033849681757359, 'mcc': 0.6611288486724195, 'f1': 0.7166332665330661, 'precision': 0.8046804680468047, 'recall': 0.6459537572254336}

Performance metrics:
{"Cohen's Kappa": 0.655244449122101, 'PRC AUC': 0.8281943615454804}


Computing feature importance via permutation shuffling for 55 features using 5000 rows with 5 shuffle sets...


...


Test set performance:
{'roc_auc': 0.949790282379023, 'accuracy': 0.9062409708176827, 'balanced_accuracy': 0.8319263438941042, 'mcc': 0.6957049359514148, 'f1': 0.7512456880030663, 'precision': 0.8, 'recall': 0.708092485549133}

Performance metrics:
{"Cohen's Kappa": 0.6937439164675693, 'PRC AUC': 0.8445045652214765}


These features in provided data are not utilized by the predictor and will be ignored: ['F2', 'F4', 'F6', 'F7', 'F8', 'F9', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F56', 'F57', 'F58', 'F59', 'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F75', 'F77', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'F90', 'F92', 'F93', 'F96', 'F97', 'F98', 'F99', 'F100', 'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110', 'F111', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F121', 'F122', 'F124', 'F126', 'F127', 'F128', 'F129', 'F130', 'F131', 'F132', 'F133', 'F134', 'F136', 'F137', 'F139', 'F141', 'F142', 'F143', 'F144', 'F145', 'F1


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.949790   0.950146     roc_auc        0.522650       0.226621   
1            LightGBMXT    0.948921   0.944320     roc_auc        0.247072       0.083958   
2              CatBoost    0.948913   0.947668     roc_auc        0.028169       0.016376   
3              LightGBM    0.945727   0.946653     roc_auc        0.084819       0.030362   
4               XGBoost    0.945365   0.945159     roc_auc        0.074232       0.031938   
5         LightGBMLarge    0.944137   0.942526     roc_auc        0.072588       0.024113   
6        NeuralNetTorch    0.943516   0.944848     roc_auc        0.181025       0.094321   
7       NeuralNetFastAI    0.940703   0.937262     roc_auc        0.147395       0.052820   
8      RandomForestEntr    0.926280   0.926937     roc_auc        0.213536       0.104373   
9        ExtraTreesEntr    0.924782   0.924709    

	211.77s	= Expected runtime (42.35s per shuffle set)
	150.24s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                      importance    stddev       p_value  n  p99_high       p99_low     percent
platelets_min       5.134019e-02  0.002667  8.702713e-07  5  0.056831  4.584952e-02  100.000000
platelets_max       2.765217e-02  0.003123  1.919612e-05  5  0.034083  2.122183e-02   53.860659
count_transfer      2.655037e-02  0.002513  9.517843e-06  5  0.031725  2.137595e-02   51.714583
urea_n_max          2.565006e-02  0.002238  6.883288e-06  5  0.030258  2.104219e-02   49.960977
urea_n_min          1.527547e-02  0.000842  1.105164e-06  5  0.017010  1.354105e-02   29.753433
platelets_mean      1.516880e-02  0.001557  1.313469e-05  5  0.018375  1.196298e-02   29.545660
drg_severity        4.817594e-03  0.000546  1.941146e-05  5  0.005941  3.694136e-03    9.383669
F94                 2.262591e-03  0.000446  1.720787e-04  5  0.003181  1.344378e-03    4.407056
F60                 1.767690e-03  0.000552  1.004219e-03  5  0.002904  6.318296e-04    3.443092
temp_max           

### 5.3.3 los_cat_50

In [ ]:
# Model Parameters
label = 'los_cat_50'
metric = 'roc_auc'


# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_los_50_{method}{death_tag}') ######                                                                                                   # add los_50_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_los_50_{preproc_tag}{lemma_tag}{method}{death_tag}'                                                                                 # add _los_50_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['los_cat_50'], y_pred, y_pred_proba)                                                                                                          # add_50
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_los_50_{method}{death_tag}.xlsx')                                                                                      # add los_50_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_los_50_{method}{death_tag}.xlsx')                                                                               # add los_50_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_los_50_{method}{death_tag}.xlsx')                                                                                 # add los_50_

tabular
frequency


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_los_50__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail:    

[1000]	valid_set's binary_logloss: 0.363229


	0.9176	 = Validation score   (roc_auc)
	4.65s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: LightGBM ...
	0.915	 = Validation score   (roc_auc)
	2.12s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8924	 = Validation score   (roc_auc)
	6.82s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8943	 = Validation score   (roc_auc)
	6.37s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.917	 = Validation score   (roc_auc)
	24.02s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8872	 = Validation score   (roc_auc)
	1.9s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8877	 = Validation score   (roc_auc)
	2.21s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 9: early stopping
	0.9134	 = Validation score   (roc_auc)
	31


Test set performance:
{'roc_auc': 0.9196277236563679, 'accuracy': 0.8403640566310315, 'balanced_accuracy': 0.8403640566310315, 'mcc': 0.6810536646164599, 'f1': 0.8378576669112252, 'precision': 0.8512224209898629, 'recall': 0.8249060965039006}

Performance metrics:
{"Cohen's Kappa": 0.6807281132620631, 'PRC AUC': 0.9255870729258632}


...


Test set performance:
{'roc_auc': 0.9258821268502181, 'accuracy': 0.8423865934700954, 'balanced_accuracy': 0.8423865934700954, 'mcc': 0.6850312977873978, 'f1': 0.8401933499340852, 'precision': 0.8520499108734403, 'recall': 0.8286622363478764}

Performance metrics:
{"Cohen's Kappa": 0.6847731869401907, 'PRC AUC': 0.9315591234519307}


These features in provided data are not utilized by the predictor and will be ignored: ['F1', 'F2', 'F3', 'F4', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F29', 'F30', 'F31', 'F33', 'F34', 'F35', 'F36', 'F37', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F48', 'F49', 'F50', 'F51', 'F52', 'F54', 'F55', 'F56', 'F57', 'F58', 'F59', 'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F75', 'F76', 'F77', 'F79', 'F80', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F90', 'F91', 'F92', 'F93', 'F94', 'F95', 'F96', 'F97', 'F98', 'F99', 'F100', 'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110', 'F111', 'F112', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F121', 'F123', 'F124', 'F126', 'F127', 'F128', 'F129', 'F130', 'F131', 'F132', 'F133', 'F134', 'F135', 'F136', 'F138', 'F139', 'F140', 'F141',


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.925882   0.926636     roc_auc        0.771404       0.287464   
1              CatBoost    0.923972   0.922226     roc_auc        0.026023       0.014763   
2              LightGBM    0.922037   0.920859     roc_auc        0.067495       0.023723   
3            LightGBMXT    0.921897   0.921815     roc_auc        0.181637       0.064414   
4               XGBoost    0.920695   0.919314     roc_auc        0.084977       0.029991   
5         LightGBMLarge    0.920291   0.921020     roc_auc        0.132311       0.040284   
6        NeuralNetTorch    0.919497   0.920278     roc_auc        0.186467       0.087908   
7       NeuralNetFastAI    0.914102   0.916471     roc_auc        0.169808       0.055621   
8      RandomForestEntr    0.897121   0.900290     roc_auc        0.254645       0.114281   
9      RandomForestGini    0.895903   0.898436    

	310.85s	= Expected runtime (62.17s per shuffle set)
	205.26s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                    importance    stddev       p_value  n  p99_high       p99_low     percent
platelets_max         0.091324  0.003051  1.493513e-07  5  0.097607  8.504085e-02  100.000000
platelets_min         0.075449  0.002678  1.901198e-07  5  0.080963  6.993483e-02   82.616687
urea_n_max            0.041249  0.001236  9.656034e-08  5  0.043794  3.870485e-02   45.168312
count_transfer        0.040821  0.001343  1.403958e-07  5  0.043586  3.805578e-02   44.699305
urea_n_min            0.021242  0.000915  4.125796e-07  5  0.023127  1.935789e-02   23.260619
platelets_mean        0.005226  0.000730  4.451180e-05  5  0.006729  3.723172e-03    5.722742
drg_severity          0.005048  0.000836  8.721172e-05  5  0.006770  3.325893e-03    5.527584
F125                  0.003817  0.000337  7.200909e-06  5  0.004511  3.123644e-03    4.179849
icustay_count         0.003611  0.000563  6.846064e-05  5  0.004769  2.452389e-03    3.953614
type_stay             0.003114  0.00048

### 5.3.4 In Hospital Mortality

In [ ]:
# Model Parameters
label = 'mortality'
metric = 'roc_auc'


# Iterate over our main methods of vectorization
vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}


for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_mortality_{method}{death_tag}') ######                                                   # add mortality_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_mortality_{preproc_tag}{lemma_tag}{method}{death_tag}'                                  # add _mortality_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['mortality'], y_pred, y_pred_proba)                                                        # change mortality
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_mortality_{method}{death_tag}.xlsx')                                     # add mortality_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_mortality_{method}{death_tag}.xlsx')                             # add mortality_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_mortality_{method}{death_tag}.xlsx')                               # add mortality_

tabular
frequency


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_mortality__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail: 


Test set performance:
{'roc_auc': 0.9524056169429098, 'accuracy': 0.9413464316671483, 'balanced_accuracy': 0.7903772783284584, 'mcc': 0.6721954558539195, 'f1': 0.6928895612708018, 'precision': 0.8267148014440433, 'recall': 0.5963541666666666}

Performance metrics:
{"Cohen's Kappa": 0.6614033354457984, 'PRC AUC': 0.8044754641610226}


Computing feature importance via permutation shuffling for 52 features using 5000 rows with 5 shuffle sets...



Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.952406   0.934858     roc_auc        0.582868       0.214623   
1            LightGBMXT    0.950487   0.928718     roc_auc        0.088573       0.025573   
2              CatBoost    0.948634   0.927960     roc_auc        0.045785       0.012605   
3         LightGBMLarge    0.946384   0.920198     roc_auc        0.059436       0.018329   
4              LightGBM    0.946263   0.924564     roc_auc        0.065969       0.019173   
5               XGBoost    0.944755   0.925061     roc_auc        0.060608       0.029594   
6        NeuralNetTorch    0.944420   0.926061     roc_auc        0.100951       0.050633   
7       NeuralNetFastAI    0.940472   0.923339     roc_auc        0.149490       0.057785   
8      RandomForestEntr    0.932532   0.910894     roc_auc        0.238013       0.101119   
9      RandomForestGini    0.929707   0.902715    

...


Test set performance:
{'roc_auc': 0.9864174131323801, 'accuracy': 0.9680728113262063, 'balanced_accuracy': 0.8999939064023399, 'mcc': 0.8328247386889346, 'f1': 0.8495575221238938, 'precision': 0.8901569186875892, 'recall': 0.8125}

Performance metrics:
{"Cohen's Kappa": 0.8317404854239292, 'PRC AUC': 0.924540963285718}


These features in provided data are not utilized by the predictor and will be ignored: ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F13', 'F14', 'F15', 'F16', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F39', 'F40', 'F41', 'F42', 'F43', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F58', 'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F75', 'F76', 'F77', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84', 'F85', 'F86', 'F88', 'F89', 'F90', 'F91', 'F92', 'F93', 'F94', 'F95', 'F96', 'F97', 'F98', 'F99', 'F100', 'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F108', 'F109', 'F110', 'F111', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F121', 'F122', 'F124', 'F125', 'F126', 'F127', 'F128', 'F129', 'F130', 'F131', 'F132', 'F133', 'F134', 'F135', 'F136', 'F137', 'F140', 'F141', 'F142', '


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.986417   0.980977     roc_auc        0.677385       0.287342   
1              LightGBM    0.985939   0.977223     roc_auc        0.049929       0.020569   
2              CatBoost    0.985882   0.977291     roc_auc        0.021582       0.013074   
3            LightGBMXT    0.985878   0.979628     roc_auc        0.092737       0.031173   
4               XGBoost    0.985037   0.977015     roc_auc        0.070807       0.026409   
5        NeuralNetTorch    0.984228   0.976418     roc_auc        0.167472       0.086905   
6         LightGBMLarge    0.983788   0.972785     roc_auc        0.045860       0.020401   
7       NeuralNetFastAI    0.980215   0.973757     roc_auc        0.140003       0.053667   
8        ExtraTreesEntr    0.979817   0.968628     roc_auc        0.249475       0.101774   
9        ExtraTreesGini    0.979472   0.969603    

	254.57s	= Expected runtime (50.91s per shuffle set)
	138.63s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                      importance    stddev       p_value  n  p99_high   p99_low     percent
F12                 5.955958e-02  0.001440  4.095193e-08  5  0.062524  0.056595  100.000000
F266                1.115353e-02  0.001408  2.983910e-05  5  0.014053  0.008254   18.726672
sysbp_min           4.600401e-03  0.000132  8.101648e-08  5  0.004872  0.004329    7.724032
F214                4.026615e-03  0.001159  7.405437e-04  5  0.006413  0.001640    6.760651
urea_n_min          2.993495e-03  0.000283  9.525364e-06  5  0.003577  0.002410    5.026052
F147                2.299688e-03  0.000623  5.891743e-04  5  0.003583  0.001016    3.861155
drg_severity        2.232898e-03  0.000654  7.913274e-04  5  0.003580  0.000886    3.749016
hr_min              1.617677e-03  0.000322  1.781426e-04  5  0.002280  0.000955    2.716065
age                 1.418772e-03  0.000150  1.489121e-05  5  0.001728  0.001109    2.382105
F236                1.252513e-03  0.000491  2.338444e-03  5

### 5.3.5 readm_cat_30

In [ ]:
# Model Parameters
label = 'readm_cat_30'
metric = 'roc_auc'

# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_readm_30_{method}{death_tag}') ######                                                        # add readm_30_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_readm_30_{preproc_tag}{lemma_tag}{method}{death_tag}'                                        # add _readm_30_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)


        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['readm_cat_30'], y_pred, y_pred_proba)                                                              # change readm_cat_30
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_readm_30_{method}{death_tag}.xlsx')                                             # add readm_30_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_readm_30_{method}{death_tag}.xlsx')                                     # add readm_30_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_readm_30_{method}{death_tag}.xlsx')                                       # add readm_30_

tabular
frequency


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_readm_30__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail:  


Test set performance:
{'roc_auc': 0.778450307514776, 'accuracy': 0.8585668881826062, 'balanced_accuracy': 0.5321114084466264, 'mcc': 0.19708617636628264, 'f1': 0.9230648330058939, 'precision': 0.8601347393087287, 'recall': 0.9959301339664236}

Performance metrics:
{"Cohen's Kappa": 0.1027906990291827, 'PRC AUC': 0.9508141277937738}


Computing feature importance via permutation shuffling for 53 features using 5000 rows with 5 shuffle sets...



Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.778450   0.790553     roc_auc        1.686980       0.540637   
1               XGBoost    0.775610   0.782262     roc_auc        0.047654       0.022288   
2              CatBoost    0.775130   0.782615     roc_auc        0.032239       0.010030   
3              LightGBM    0.773361   0.772182     roc_auc        0.031071       0.012040   
4         LightGBMLarge    0.773245   0.773855     roc_auc        0.052679       0.017833   
5            LightGBMXT    0.771061   0.773678     roc_auc        0.056587       0.017780   
6      RandomForestEntr    0.748707   0.765836     roc_auc        0.254020       0.111953   
7        NeuralNetTorch    0.732253   0.748340     roc_auc        0.089398       0.047136   
8      RandomForestGini    0.731284   0.739034     roc_auc        0.320732       0.101847   
9        ExtraTreesEntr    0.729930   0.754069    

...


Test set performance:
{'roc_auc': 0.7974594771214797, 'accuracy': 0.8616006934412019, 'balanced_accuracy': 0.5548488069584783, 'mcc': 0.24682950231193018, 'f1': 0.9242208511311502, 'precision': 0.8661230541141587, 'recall': 0.9906732236730541}

Performance metrics:
{"Cohen's Kappa": 0.1666522984786225, 'PRC AUC': 0.9564194323466768}


These features in provided data are not utilized by the predictor and will be ignored: ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F28', 'F30', 'F31', 'F33', 'F34', 'F36', 'F39', 'F40', 'F41', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F57', 'F58', 'F59', 'F61', 'F62', 'F63', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F75', 'F76', 'F77', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'F90', 'F91', 'F92', 'F93', 'F94', 'F95', 'F96', 'F97', 'F98', 'F99', 'F100', 'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110', 'F111', 'F112', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F121', 'F122', 'F123', 'F124', 'F126', 'F127', 'F128', 'F129', 'F130', 'F131', 'F132', 'F133', 'F134', 'F135', 'F136', 'F137', 'F138', 'F139', 


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.797459   0.800612     roc_auc        0.660514       0.253254   
1              CatBoost    0.793015   0.790133     roc_auc        0.021155       0.011941   
2         LightGBMLarge    0.791984   0.790525     roc_auc        0.047876       0.018573   
3               XGBoost    0.789879   0.793448     roc_auc        0.061965       0.024213   
4              LightGBM    0.789823   0.789284     roc_auc        0.028771       0.013082   
5            LightGBMXT    0.783322   0.778498     roc_auc        0.044996       0.016596   
6      RandomForestEntr    0.768994   0.761790     roc_auc        0.220511       0.102957   
7        NeuralNetTorch    0.758019   0.757007     roc_auc        0.162408       0.080691   
8       NeuralNetFastAI    0.755592   0.759194     roc_auc        0.140088       0.049978   
9      RandomForestGini    0.754245   0.752647    

	237.95s	= Expected runtime (47.59s per shuffle set)
	192.22s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                      importance    stddev   p_value  n  p99_high   p99_low     percent
age                 4.063324e-02  0.003500  0.000007  5  0.047840  0.033426  100.000000
F125                2.467211e-02  0.001488  0.000002  5  0.027736  0.021608   60.719027
F293                1.875706e-02  0.002662  0.000047  5  0.024237  0.013277   46.161857
ethnicity           1.108886e-02  0.001367  0.000027  5  0.013904  0.008274   27.290124
icd_chapter         8.736282e-03  0.003250  0.001928  5  0.015428  0.002045   21.500332
religion            5.844199e-03  0.001990  0.001392  5  0.009942  0.001746   14.382803
marital_status      4.741285e-03  0.002315  0.005092  5  0.009508 -0.000025   11.668488
insurance           3.291619e-03  0.001180  0.001683  5  0.005721  0.000862    8.100803
temp_mean           2.517003e-03  0.001216  0.004903  5  0.005020  0.000014    6.194443
calcium_min         2.126642e-03  0.000591  0.000646  5  0.003343  0.000911    5.233748
F32        

### 5.3.6 readm_cat_365

In [ ]:
# Model Parameters
label = 'readm_cat_365'
metric = 'roc_auc'

# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_readm_365_{method}{death_tag}') ######                                                        # add readm_30_
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text_readm_365_{preproc_tag}{lemma_tag}{method}{death_tag}'                                        # add _readm_30_
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)


        # evaluate performance on the test set
        per_tab = predictor.evaluate(test.copy())                               # change 2nd: test-->test.copy()
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['readm_cat_365'], y_pred, y_pred_proba)                                                              # change readm_cat_30
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_readm_365_{method}{death_tag}.xlsx')                                             # add readm_30_
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_readm_365_{method}{death_tag}.xlsx')                                     # add readm_30_
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_readm_365_{method}{death_tag}.xlsx')                                       # add readm_30_

tabular
frequency


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC_IV/models/text_readm_365__preproc_heavierfrequency"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Memory Avail: 


Test set performance:
{'roc_auc': 0.7934591230278107, 'accuracy': 0.8815371279976886, 'balanced_accuracy': 0.5228906131818742, 'mcc': 0.16433738011930485, 'f1': 0.09090909090909091, 'precision': 0.7068965517241379, 'recall': 0.0485781990521327}

Performance metrics:
{"Cohen's Kappa": 0.07642692336480217, 'PRC AUC': 0.3761167401794483}


Computing feature importance via permutation shuffling for 53 features using 5000 rows with 5 shuffle sets...



Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.793459   0.791432     roc_auc        1.447210       0.536925   
1              CatBoost    0.787599   0.782821     roc_auc        0.019912       0.011147   
2              LightGBM    0.785929   0.781813     roc_auc        0.039529       0.017510   
3         LightGBMLarge    0.785072   0.780319     roc_auc        0.071542       0.019683   
4            LightGBMXT    0.783628   0.779403     roc_auc        0.067672       0.021839   
5               XGBoost    0.782361   0.781729     roc_auc        0.059395       0.021008   
6      RandomForestEntr    0.760847   0.764287     roc_auc        0.245695       0.101961   
7        NeuralNetTorch    0.753922   0.744794     roc_auc        0.103408       0.052942   
8       NeuralNetFastAI    0.751472   0.742418     roc_auc        0.154265       0.044978   
9        ExtraTreesEntr    0.750547   0.745860    

...


Test set performance:
{'roc_auc': 0.8057287256190846, 'accuracy': 0.8834151979196764, 'balanced_accuracy': 0.5392646386860233, 'mcc': 0.21022863754764523, 'f1': 0.14963119072708114, 'precision': 0.6761904761904762, 'recall': 0.08412322274881516}

Performance metrics:
{"Cohen's Kappa": 0.12605084876758088, 'PRC AUC': 0.39607662135339605}


These features in provided data are not utilized by the predictor and will be ignored: ['F1', 'F2', 'F3', 'F4', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F13', 'F14', 'F15', 'F16', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F39', 'F40', 'F41', 'F42', 'F43', 'F45', 'F46', 'F47', 'F48', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56', 'F58', 'F59', 'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F73', 'F75', 'F76', 'F77', 'F78', 'F79', 'F80', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'F90', 'F91', 'F92', 'F93', 'F94', 'F95', 'F96', 'F98', 'F100', 'F101', 'F102', 'F103', 'F104', 'F106', 'F107', 'F108', 'F109', 'F110', 'F111', 'F112', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120', 'F122', 'F124', 'F126', 'F127', 'F128', 'F129', 'F130', 'F131', 'F132', 'F133', 'F134', 'F135', 'F136', 'F138', 'F140', 'F141', 'F142', 'F143', 'F144', 'F145', 'F146


Model Leaderboard:
                  model  score_test  score_val eval_metric  pred_time_test  pred_time_val  \
0   WeightedEnsemble_L2    0.805729   0.810368     roc_auc        2.277177       0.677472   
1              CatBoost    0.804727   0.799594     roc_auc        0.024921       0.013371   
2              LightGBM    0.798800   0.804008     roc_auc        0.049095       0.018893   
3               XGBoost    0.797754   0.800360     roc_auc        0.075790       0.030322   
4         LightGBMLarge    0.796434   0.803882     roc_auc        0.050341       0.017237   
5            LightGBMXT    0.794331   0.793363     roc_auc        0.091597       0.029530   
6      RandomForestEntr    0.776980   0.773347     roc_auc        0.238713       0.114540   
7        NeuralNetTorch    0.774557   0.760668     roc_auc        0.203087       0.112225   
8       NeuralNetFastAI    0.772266   0.763240     roc_auc        0.139552       0.049996   
9        ExtraTreesEntr    0.762198   0.768152    

	805.66s	= Expected runtime (161.13s per shuffle set)
	490.73s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                    importance    stddev   p_value  n  p99_high   p99_low     percent
age                   0.031771  0.003758  0.000023  5  0.039510  0.024033  100.000000
F125                  0.022841  0.005313  0.000327  5  0.033781  0.011900   71.891448
F44                   0.020829  0.003177  0.000063  5  0.027371  0.014288   65.560167
icd_chapter           0.020477  0.005118  0.000432  5  0.031016  0.009939   64.452005
F74                   0.012337  0.006053  0.005179  5  0.024800 -0.000126   38.831540
ethnicity             0.011751  0.002265  0.000158  5  0.016414  0.007088   36.986964
platelets_min         0.011398  0.001492  0.000034  5  0.014471  0.008325   35.874942
temp_mean             0.005290  0.003489  0.013757  5  0.012473 -0.001893   16.649465
urea_n_min            0.004648  0.001926  0.002854  5  0.008613  0.000682   14.628606
religion              0.004455  0.000736  0.000086  5  0.005971  0.002940   14.023586
F190                  0.004272  0